### Stratégie :
On récupère le point de départ, on le région growth.
On fait une recherche sur la couche du dessous sur le centre trouvé précédemment puis en région growing sur les plus proches voisins jusqu'à trouver un point d'intensité similaire +-5% puis on le region growth. On limite le region growing à la taille de la tache trouvée sur la précédente couche + 1 voisin
Si on ne trouve rien dans cette zone, on met un False dans la liste et on descend d'une couche avec une tolérance d'intensité de +1% et d'un voisin supplémentaire 

In [1]:

import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import cv2
from tqdm.notebook import tqdm
from napari.utils.colormaps import colormap_utils as cu
from functools import partial
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
from networkclass import *
from functions.segmentation_utils import *

In [3]:
image_array = np.load(Path("temp/preprocessed_image.npy"))

target = np.array(extract_centroid_first_slice(image_array)).astype(np.uint8)
network_engine = NetworkEngine(image_array,len(image_array)-1)

network_engine.network_manager.set_branch_target(0,target)
network_engine.network_manager.get_nested_intensity(0).add_number(image_array[len(image_array)-1,target[1],target[0]])
network_engine.network_manager.get_mean_radius(0).add_number(10)
network_engine.run()

knee_depth, knee_contour = find_knee_depth(image_array)
vessel1,vessel2 = extract_arteries_position_from_knee(image_array[knee_depth],knee_contour)
# plt.plot([vessel1[0]],[vessel1[1]],marker='o', color='r')
# plt.plot([vessel2[0]],[vessel2[1]],marker='o', color='r')
# plt.imshow(image_array[knee_depth])
# plt.show()
network_engine.explore_reverse(vessel1,knee_depth)
network_engine.explore_reverse(vessel2,knee_depth)
network_engine.force_prepare_explore(vessel1,knee_depth)
network_engine.force_prepare_explore(vessel2,knee_depth)
network_engine.run()

  0%|          | 0/1331 [00:00<?, ?it/s]

[257.2177589852009, 285.5137420718816]


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:

import gzip
import napari
# mask,mask2 = draw_network(mask,mask2,tube_network,index,True)
mask_list = network_engine.network_manager.generate3DImages(image_array.shape)

SHOW_IN_NAPARI = False

if SHOW_IN_NAPARI:
    viewer = napari.Viewer()
    image_layer = viewer.add_image(image_array)
    colorindex = 0
    for mask in mask_list:
        image_layer = viewer.add_image(mask,colormap=list(cu.AVAILABLE_COLORMAPS.keys())[colorindex%5],blending="additive")
        colorindex+=1
        print(np.sum(mask))

    # start the event loop and show the viewer
    napari.run()

[0, 1, 4, 5, 6, 7, 8, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
[231, 30, 0, 0, 195, 151, 95, 21, 39, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 67, 321, 427, 377, 202, 322, 974, 974, 37, 51, 320, 307]


In [ ]:
result_network_manager = network_engine.merge_network()
mask_list_merged = result_network_manager.generate3DImages(image_array.shape)


border collapse
30 merged to 25
border collapse
30 merged to 25
border collapse
30 merged to 25
border collapse
30 merged to 25
border collapse
30 merged to 25
border collapse
30 merged to 25
border collapse
30 merged to 25
border collapse
30 merged to 25
border collapse
30 merged to 25
border collapse
30 merged to 25
border collapse
30 merged to 25
border collapse
30 merged to 25
border collapse
30 merged to 25
border collapse
30 merged to 25
border collapse
30 merged to 25
border collapse
30 merged to 25
border collapse
30 merged to 25
border collapse
30 merged to 25
border collapse
30 merged to 25
border collapse
30 merged to 25
border collapse
30 merged to 25
border collapse
30 merged to 25
border collapse
30 merged to 25
border collapse
30 merged to 25
border collapse
30 merged to 25
border collapse
30 merged to 25
border collapse
30 merged to 25
border collapse
30 merged to 25
border collapse
30 merged to 25
border collapse
30 merged to 25
border collapse
30 merged to 25
border c

In [ ]:
SHOW_IN_NAPARI = True

if SHOW_IN_NAPARI:
    viewer = napari.Viewer()
    image_layer = viewer.add_image(image_array)
    colorindex = 0
    for mask in mask_list_merged:
        if np.sum(mask) > 0:
            image_layer = viewer.add_image(mask,colormap=list(cu.AVAILABLE_COLORMAPS.keys())[colorindex%5],blending="additive")
            colorindex+=1

    # start the event loop and show the viewer
    napari.run()

f = gzip.GzipFile("temp/mask_list.npy.gz", "w")
np.save(file=f, arr= mask_list[:-1])#On retire le debug
f.close()